In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# If using a remote server it is neccessary to navigate to your workspace
import os 

os.chdir('/content/drive/MyDrive/BiasMitigation/LNTL')
print('Directory changed to...',  os.getcwd())

Directory changed to... /content/drive/MyDrive/BiasMitigation/LNTL


In [ ]:
# import torch
# import platform 
# from platform import python_version

# print(python_version())

# print(torch.__version__)
# print(platform.platform())

In [3]:
# -*- coding: utf-8 -*-

# file imports
! pip install import-ipynb 
import import_ipynb
import random
#from option import get_option
from trainer_merger import *
from utils.utils_LNTL import save_option
from data_loaders.data_loader_cityscapes import *
from data_loaders.data_loader_SYNTHIA import *

# torch imports
import torch
from torch.backends import cudnn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.utils.data as data

# argparse jupyter workaround
from argparse import Namespace
# manually add the options becuase using juypter not command line:
# Must make it a Namespace class to be used as the argparser would be
root = '/content/drive/MyDrive/BiasMitigation/'

args = Namespace(exp_name = 'Exp2_Stage1', # Enter in format Exp1_Stage2
                 batch_size = 4,
                 momentum = 0.9,
                 optimiser = 'ADAM',  # accepts either ADAM or SGD
                 lr = 0.0001,  # try lr = 0.0001 for ADAM and lr = 0.001 SGD 
                 lr_decay_rate = 0.1,
                 lr_decay_period = 40, # 40 for cityscapes, 20 for synthia
                 weight_decay = 0.0001, #arbitrary value off net doubt we have time to tune. would like some paper to back up this being a good value
                 max_step = 100,  # 100 for cityscapes and 50 for synthia. 
                 seed = 2,
                 ##############
                 log_step = 100 , # 100 for cityscapes and 200 for SYNTHIA
                 save_step = 1, # save every epoch
                 dataset = 'SYNTHIA', # Accepts 'Cityscapes' or 'SYNTHIA' only
                 save_dir = root + 'LNTL/training_logs',
                 train_baseline = False,
                 train_greyscale = False,
                 train_bias_head_only = False,
                 #############
                 checkpoint =  None, # root + 'LNTL/training_logs/Deeplab/Cityscapes/baseline/colour/Exp7_Stage1/checkpoints/checkpoint_epoch_0100.pth', 
                 load_bias_head = False,
                 load_seg_head = False,
                 #############
                 val_only_greyscale = False,
                 val_only_jitter = True,
                 val_only_invert = False,
                 #############
                 random_seed = None,
                 num_workers = 2,
                 cudnn_benchmark = True,
                 #############
                 cuda = True ,
                 debug = False,  
                 #############
                 network_type = 'Deeplab',  # Enter only Deeplab or SegNet
                 notes = 'Colour Jitter in the Val set for LNTL scheme with SYNTHIA'
                 )

####~~~~

def log_file_setup(option):
    # setting up training log files:
    if option.train_baseline:
        if option.train_greyscale:
            kind = 'baseline_greyscale'
            path = option.save_dir + '/' + option.network_type + '/' + option.dataset + '/baseline' + '/greyscale' + '/' + option.exp_name 
            if not os.path.exists(path):
                os.makedirs(path)
        else:
            kind = 'baseline_colour'
            # path = os.path.join(option.save_dir, '/' , option.network_type, '/', option.dataset, '/baseline', '/colour', '/', option.exp_name )
            path = option.save_dir + '/' + option.network_type + '/' + option.dataset + '/baseline' + '/colour' + '/' + option.exp_name
            if not os.path.exists(path):
                os.makedirs(path)
    else:
        kind = 'LNTL'
        #path = os.path.join(option.save_dir, '/' , option.network_type, '/', option.dataset, '/LNTL', '/', option.exp_name )
        path = option.save_dir + '/' + option.network_type + '/' + option.dataset + '/LNTL' + '/' + option.exp_name
        if not os.path.exists(path):
            os.makedirs(path)
    
    return path


def backend_setting(option):

    if option.random_seed is None:
        option.random_seed = random.randint(1,10000)
    torch.manual_seed(option.random_seed)

    if torch.cuda.is_available() and not option.cuda:
        print('WARNING: GPU is available, but we are not using it')

    if not torch.cuda.is_available() and option.cuda:
        option.cuda = False
        print('Warning: Youve asked for CUDA but no GPU is available...setting CUDA to False')

    if option.cuda:
        os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
        #os.environ['CUDA_VISIBLE_DEVICES'] = ','.join([str(i) for i in option.gpu_ids])
        torch.cuda.manual_seed_all(option.random_seed)
        cudnn.benchmark = option.cudnn_benchmark
    
    if option.train_baseline:
        option.is_train = True


def main():
    #option = get_option()
    option = args
    #debug:
    #print(option)
    
    path = log_file_setup(option)
    backend_setting(option)
    trainer = Trainer(option, path)

    if option.dataset == 'Cityscapes':
        train_dataset = DatasetTrain(option)
        val_dataset = DatasetVal(option)
    elif option.dataset == 'SYNTHIA':
        train_dataset = DatasetTrain_SYNTHIA(option)
        val_dataset = DatasetVal_SYNTHIA(option)


    train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                                batch_size=option.batch_size,
                                                shuffle=True,
                                                num_workers=option.num_workers)

    val_loader = torch.utils.data.DataLoader(dataset = val_dataset,
                                                batch_size=option.batch_size,
                                                shuffle=False,
                                                num_workers=option.num_workers)

    #if option.is_train:   
    save_option(option, path)
    trainer.train(train_loader = train_loader, val_loader = val_loader)
    #else:
    #   trainer._validate(trainval_loader) # I dont think we need this

        #pass

if __name__ == '__main__': main()

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=57c363e18cb9af54f270b0c7ba2acf4f26923f47f15a184f4873f6ffb3d98e9f
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb
importing Jupyter notebook from trainer_merger.ipynb
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Changed the local path to.... /content/drive/MyDrive/BiasMitigation/LNTL
importing Jupyter notebook from /content/drive/My Drive/BiasMitigation/LNTL/models/Deeplab/deeplabv3.ipynb
importing Jupyter notebook from /content/drive/My Drive/BiasMitigation/LNTL/models/Deeplab/resnet.ipynb
importing Jupyter notebook from /content/drive/My Drive/BiasMitigation/LNTL/models/Deeplab/aspp.ipynb
importing Jupyter notebook from /content/drive/My Drive/BiasMitigation/LNTL/models/SegNet/segnet.ipynb
importing Jupyter notebook fr

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3487: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details

Training epoch loss:.... 1.5604243


[Exp2_Stage1] INFO: [VAL] cls loss : 1.683854 (epoch 0.00)
[Exp2_Stage1] INFO: [VAL] cls loss : 1.813755 (epoch 0.80)


Validation epoch loss:.... 1.7552792
Checkpoint Saved. Epoch : 0


...

Validation epoch loss:.... 1.5186121
Checkpoint Saved. Epoch : 98


[Exp2_Stage1] INFO: [TRAIN] cls loss : 0.228299 (epoch 99.00)
[Exp2_Stage1] INFO: [TRAIN] cls loss : 0.273481 (epoch 99.13)
[Exp2_Stage1] INFO: [TRAIN] cls loss : 0.278394 (epoch 99.26)
[Exp2_Stage1] INFO: [TRAIN] cls loss : 0.425798 (epoch 99.40)
[Exp2_Stage1] INFO: [TRAIN] cls loss : 0.418078 (epoch 99.53)
[Exp2_Stage1] INFO: [TRAIN] cls loss : 0.350146 (epoch 99.66)
[Exp2_Stage1] INFO: [TRAIN] cls loss : 0.378899 (epoch 99.80)
[Exp2_Stage1] INFO: [TRAIN] cls loss : 0.364546 (epoch 99.93)


Training epoch loss:.... 0.33159688


[Exp2_Stage1] INFO: [VAL] cls loss : 1.501502 (epoch 99.00)
[Exp2_Stage1] INFO: [VAL] cls loss : 1.178709 (epoch 99.80)


Validation epoch loss:.... 1.487923
Checkpoint Saved. Epoch : 99


In [ ]:
#!/usr/local/cuda/bin/nvcc --version

#!nvidia-smi 

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
Tue Aug 17 09:24:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |         

In [ ]:
import os

print( len(os.listdir('/content/drive/MyDrive/BiasMitigation/Datasets/CityscapesRainVal/colour_labels') ) )
print( len(os.listdir('/content/drive/MyDrive/BiasMitigation/Datasets/CityscapesRainVal/gt_labels') ) )
print( len(os.listdir('/content/drive/MyDrive/BiasMitigation/Datasets/CityscapesRainVal/images') ) )

295
295
0


In [ ]:
print( len(os.listdir('/content/drive/MyDrive/BiasMitigation/Datasets/CityscapesFogVal/colour_labels') ) )
print( len(os.listdir('/content/drive/MyDrive/BiasMitigation/Datasets/CityscapesFogVal/gt_labels') ) )
print( len(os.listdir('/content/drive/MyDrive/BiasMitigation/Datasets/CityscapesFogVal/images') ) )

550
550
0
